# Build Chains and Results  
Instructions: https://fsworks.atlassian.net/wiki/spaces/~558841686/pages/edit-v2/2087550977

In [4]:
instance_name = 'services_dev'
data_file_name = 'EMS_DCMA_DD_23.08.graphml'
experiment = 'pre_merge_test'
tasks_types = 'tdas'
results = 'prt'
query = 'code'
if not experiment: experiment = data_file_name.replace('.graphml', '').replace('.', '_')
from modules.run_service import service_manager
service_manager(instance_name, data_file_name, experiment, tasks_types, results, query)

service started on 06:26:28
EC2 instance services_dev i-0249408ea16bc730b started
The instance is running
Data file uploaded to S3
Run attempt encountered error: []
Calculation finished
EC2 instance stopped
Preparing results
bucket_chains_path, chains_path: pre_merge_test/chains.parquet pre_merge_test/chains.parquet
results_bucket, prt_path: chainsresults pre_merge_test/prt
prt_files: ['results_copy_0.parquet']
The following results files are available in the pre_merge_test directory:
 chains.parquet,results.xlsx,prt
12370 chains written to chains.parquet
200924 tasks rows written to parquet files in the prt sub-directory
service started on 06:26:28
service ended on 06:30:00


# Query Results  

In [2]:
import duckdb
import os
con = duckdb.connect()
prt_files_path = os.path.join(experiment, 'prt')
results_files = "'{e}/*.parquet'".format(e=prt_files_path)
query = """SELECT * FROM {rf} LIMIT 10""".format(rf=results_files)
query_df = con.execute(query).df()
# To save the query results run:
#query_df.to_excel('query_df.xlsx')
display(query_df)

,ID,ChainID,TaskID,NeighbourID,Dependency,TaskType,Label,PlannedStart,PlannedEnd,ActualStart,ActualEnd,Float1,Status,File,planned_duration,actual_duration
0,MWH.06.M1000,C1500001,C1500001T1,MWH06-9914,FS,TT_Mile,Project Start,2020-04-27,2020-04-27,2020-02-24,2020-02-24,0,TK_Complete,MWH-06-UP#13_FSW_REV.graphml,0,0.0
1,MWH06-9914,C1500001,C1500001T2,MWH06.C3.C1010,FS,TT_Task,#1. Rain And High Winds- 10/13/2020(half day only),2021-02-01,2021-02-01,2020-10-13,2020-10-13,0,TK_Complete,MWH-06-UP#13_FSW_REV.graphml,0,0.0
2,MWH06.C3.C1010,C1500001,C1500001T3,MWH06.C3.C1050,FS,TT_Task,Electrical Underground Rough-In,2020-02-11,2020-11-16,2020-12-10,2020-10-21,0,TK_Complete,MWH-06-UP#13_FSW_REV.graphml,14,9.0
3,MWH06.C3.C1050,C1500001,C1500001T4,MWH06.C3.C1110,FS,TT_Task,Exterior Equipment Pads - Electrical,2021-02-01,2021-02-01,2020-11-30,2020-11-30,0,TK_Complete,MWH-06-UP#13_FSW_REV.graphml,0,0.0
4,MWH06.C3.C1110,C1500001,C1500001T5,MWH06-10420,FS,TT_Task,Set Exterior Electrical OFCI Equipment - GEN,2021-01-03,2021-05-03,2021-03-18,2021-03-18,0,TK_Complete,MWH-06-UP#13_FSW_REV.graphml,4,0.0
5,MWH06-10420,C1500001,C1500001T6,MWH06-10359,FS,TT_Task,COLO-3 Cell-1 Install Missing Raceway Bussing within generator,2021-04-26,2021-04-28,2021-05-04,2021-08-04,0,TK_Complete,MWH-06-UP#13_FSW_REV.graphml,2,3.0
6,MWH06-10359,C1500001,C1500001T7,MWH06-10569,FS,TT_Task,Generator Utility Power Pull,2021-05-19,2021-05-24,2021-08-05,2021-05-13,0,TK_Complete,MWH-06-UP#13_FSW_REV.graphml,5,5.0
7,MWH06-10569,C1500001,C1500001T8,MWH06.C3.C1380,FS,TT_Task,COLO-3 CEll-1 Reinstall tray in MER,2021-06-28,2021-06-30,2021-05-14,2021-05-19,0,TK_Complete,MWH-06-UP#13_FSW_REV.graphml,2,5.0
8,MWH06.C3.C1380,C1500001,C1500001T9,MWH06.C3.C1360,SS,TT_Task,Generator Utility Power Test-Term,2021-02-06,2021-10-06,2021-05-20,2021-05-31,0,TK_Complete,MWH-06-UP#13_FSW_REV.graphml,8,11.0
9,MWH06.C3.C1360,C1500001,C1500001T10,MWH06.C3.C1400,FS,TT_Task,Electrical Feeders to Mechanical Equipment Pull-Test-Term,2021-01-06,2021-06-24,2021-05-25,2021-06-18,0,TK_Complete,MWH-06-UP#13_FSW_REV.graphml,23,24.0


# Read and Write Chains  

In [ ]:
import pandas as pd
chains_path = os.path.join(experiment, 'chains.parquet')
chains_df = pd.read_parquet(chains_path)
chains = list(chains_df['Chain'])
chains = '\n'.join(chains)+'\n'
with open('chains.txt', 'w') as f: f.write(chains)
display(chains_df.head())
display(chains_df.info())